**This notebook allows to simulate data, classify it using a HLBM model and evaluate the model in a controlled environnement. The model is a semi-supervised (or constrained) LBM `HLBM` using pairwise constraints in both row and column space.**

# Imports

In [ ]:
import sys, os
os.path.dirname(sys.executable), sys.version, sys.path

In [ ]:
%config Completer.use_jedi = False
#!jt -t onedork -fs 100 -altp -tfs 11 -nfs 100 -cellw 60% -T -N
%pip list

In [ ]:
import sys
import pathlib
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.metrics import confusion_matrix

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", message='Deprecation')

from dcblockmodels.models.hlbm import HLBM

from dcblockmodels import metrics, plot, data
from dcblockmodels.models.utils import similarity_matrices, general, init
from dcblockmodels.models.utils.smoothing_schedule import SmoothingSchedule

# Data

## Sampling the data

In [ ]:
# whether we sample from a SBM or LBM
model_type = 'LBM'
# in case of SBM, whether the graph is directed
directed = True
# number of time steps
T = 10
# nb of row nodes, nb of column nodes
N, D = 100, 200
# nb row clusters, nb of column clusters 
Kz, Kw = 3, 4

In [ ]:
level_alpha = 'medium'
level_beta = 'medium'
level_pi = 'medium'
level_rho = 'medium'

alphas_dirichlet = {
    'very_easy': 10,
    'easy': 8,
    'medium': 6,
    'hard': 4
}
diag_vals = {
  'diag': 0,
  'easy': .9,
  'medium': .75,
  'hard': .6
}

alpha = data.generate_initial_proportions(Kz, alphas_dirichlet[level_alpha])
beta = data.generate_initial_proportions(Kw, alphas_dirichlet[level_beta])
prior_init = {'alpha': alpha, 'beta': beta}

pi = data.generate_diag_transition_matrix(Kz, diag_vals[level_pi]) 
rho = data.generate_diag_transition_matrix(Kw, diag_vals[level_rho])
prior_trans = {'pi': pi, 'rho': rho}

alpha, pi, beta, rho

In [ ]:
with_margins = True # True, False
constant_margins = False # True, False
start, stop, step = 1, 50, .1
order_power_law = -1.5 # margins ~ Unif(start, stop)^order_power_law
ar_margins, a_ar, sigma2_ar = True, 1.1, .05 # mu ~ AR1 : mu_{t+1} = N(a mu_t + c, sigma2) (c s.t. mu increasing if sigma2 = 0)

if with_margins:
    mu, nu = data.generate_margins(
            T, N, D, constant_margins, start, stop, step,
            directed, order_power_law,
            ar_margins, a_ar, sigma2_ar
    )
    margins = {'mu': mu, 'nu': nu}
else:
    margins = None
    
margins

In [ ]:
with_absent_nodes = False
min_proba_t = .0
max_proba_t = .2
proba_absent = None

if with_absent_nodes:
    absent_row_nodes = data.sample_absent_nodes(
        T, N,
        min_proba_t=min_proba_t,
        max_proba_t=max_proba_t,
        proba_absent=proba_absent
    )
    if not directed:
        absent_col_nodes = absent_row_nodes.copy()
    else:
        absent_col_nodes = data.sample_absent_nodes(
            T, D,
            min_proba_t=min_proba_t,
            max_proba_t=max_proba_t,
            proba_absent=proba_absent
        )
else:
    absent_row_nodes, absent_col_nodes = [], []

absent_nodes = {
    'absent_row_nodes': absent_row_nodes,
    'absent_col_nodes': absent_col_nodes
}

absent_nodes

In [ ]:
# scaling factor for the matrix gamma : determines the separability level
# lower is harder and more sparse
gamma_0 = .01

# defines the sparsity in a block
# block_sparsity_matrix[t, k, l] is the proba of a zero
# in block (k, l) at time t
# corresponds to the \beta_{kl}^t of Matias
block_sparsity_matrix = None
# block_sparsity_matrix = 0.1 * np.ones((Kz, Kw), dtype='float')

# if we add gaussian noise to the sampled graph
# not advised since it can make models with lower
# complete data log likelihood give better classification results
# than model with higher complete data log likelihood
noise_level_ = 0.


if Kz == 3 and Kw == 4:
    gamma  = gamma_0 * np.array([
        [1, 2, 3, 1 ],
        [3, 1, 2, 3 ],
        [2, 3, 1, 4 ]
    ])
elif Kz == 3 and Kw == 3:
    gamma  = gamma_0 * np.array([
        [1, 2, 3],
        [3, 1, 2],
        [2, 3, 1]
    ])
else:
    raise ValueError

if T > 1:
    gamma = np.stack([gamma for _ in range(T)], axis=0)
    if block_sparsity_matrix is not None:
        block_sparsity_matrix = np.stack([block_sparsity_matrix for _ in range(T)], axis=0)
        
gamma

In [ ]:
dimensions = {'N': N, 'D': D}
n_clusters = {'Kz': Kz, 'Kw': Kw}

self_loops = True
dtype = 'int32'

X, Z, W = data.generate_data(
    T,
    model_type,
    dimensions,
    n_clusters,
    prior_init,
    prior_trans,
    gamma,
    with_margins,
    margins,
    self_loops,
    directed,
    noise_level_,
    with_absent_nodes,
    absent_nodes,
    dtype,
    block_sparsity_matrix
)

## Plot

### Block view & link with matrix factorization

In [ ]:
t_plot = 0
X_ = X[t_plot] if X.ndim == 3 else X
Z_ = Z[t_plot] if X.ndim == 3 else Z
W_ = W[t_plot] if X.ndim == 3 else W
gamma_ = gamma[t_plot] if X.ndim == 3 else gamma

row_indices = np.argsort(Z_.astype(int))
col_indices = np.argsort(W_.astype(int))

cmap = sns.cubehelix_palette(light=1., as_cmap=True)
f, ax = plt.subplots(1, 4, figsize=(4 * 5, 5))

sns.heatmap(X_, ax=ax[0], cbar=False, square=False, xticklabels=False, yticklabels=False, cmap=cmap)
ax[0].set_title('Raw data')

sns.heatmap(X_[row_indices, :], ax=ax[1], cbar=False, square=False, xticklabels=False, yticklabels=False, cmap=cmap)
ax[1].set_title('Row-reorganized data')

sns.heatmap(X_[np.ix_(row_indices, col_indices)], ax=ax[2], cbar=False, square=False, xticklabels=False, yticklabels=False, cmap=cmap)
ax[2].set_title('Row and column-reorganized data')

Z_encoded = general.encode(Z_, Kz)
W_encoded = general.encode(W_, Kw)
X_approx = Z_encoded.dot(gamma_).dot(W_encoded.T)
sns.heatmap(X_approx[np.ix_(row_indices, col_indices)], ax=ax[3], cbar=False, square=False, xticklabels=False, yticklabels=False, cmap=cmap)
ax[3].set_title('Connectivity-approximized data')

plt.tight_layout()

### Dimensionality reduction with Correspondence Analysis

In [ ]:
t_plot = 0
t_plot = [t for t in range(T)]
t_plot = [0, T//2, T - 1]

if X.ndim == 2:
    plot.CA_plot(X, Z, W)
else:
    if type(t_plot) == int:
        t_plot = [t_plot]

    n_plots = len(t_plot)
    f, ax = plt.subplots(n_plots, 2, figsize=(10, 5 * n_plots))
    for i, t in enumerate(t_plot):
        W_plot = W[t] if W is not None else None

        absent_row = [tup[1] for tup in absent_row_nodes if tup[0] == t]
        absent_col = [tup[1] for tup in absent_col_nodes if tup[0] == t]

        plot.CA_plot(
            X[t],
            Z[t], W_plot,
            absent_row, absent_col,
            ax=ax[i]
        )
        ax[i, 0].set_title(f't = {t}')
        ax[i, 1].set_title(f't = {t}')


### True margins

Plot margins over time, in the dynamic case

In [ ]:
n_nodes = 20
f, ax = plt.subplots(1, 2, figsize=(2 * 6, 4))

ax[0].plot(margins['mu'][:, np.random.choice(N, size=n_nodes)]);
ax[0].set_title('True row margins mu');

ax[1].plot(margins['nu'][:, np.random.choice(D, size=n_nodes)]);
ax[1].set_title('True col margins nu');

### Factorial Discriminant Analysis

Measures the level of linear separability of the classes after projection onto R^N using correspondence analysis

See Discriminative Factorial Analysis : http://www.math.u-bordeaux.fr/~mchave100p/wordpress/wp-content/uploads/2013/10/AFD.pdf

In [ ]:
t_plot = 0
t_plot = [t for t in range(T)]
t_plot = [0, T//2, -1]

n_components = 3

f, ax = plt.subplots(len(t_plot), 2, squeeze=False, sharex=True, sharey=True, figsize=(5 * len(t_plot), 8))
xs = np.arange(n_components, dtype='int')

if X.ndim == 3:
    for i, t in enumerate(t_plot):
        res = metrics.AFD_CA_linear_separation(
            X[t], Z[t], W[t],
            n_components=n_components,
            absent_row_nodes=absent_row_nodes,
            absent_col_nodes=absent_col_nodes
        )

        ax[i, 0].bar(xs, res[0])
        ax[i, 1].bar(xs, res[1])
        ax[i, 0].set_xlabel('factorial axis')
        ax[i, 1].set_xlabel('factorial axis')
        ax[i, 0].set_title(f'Rows, T = {t}')
        ax[i, 1].set_title(f'Cols, T = {t}')
else:
    res = metrics.AFD_CA_linear_separation(
        X, Z, W,
        n_components=n_components,
        absent_row_nodes=absent_row_nodes,
        absent_col_nodes=absent_col_nodes
    )
    ax[0, 0].bar(xs, res[0])
    ax[0, 1].bar(xs, res[1])
    ax[0, 0].set_xlabel('factorial axis')
    ax[0, 1].set_xlabel('factorial axis')
    ax[0, 0].set_title(f'Rows')
    ax[0, 1].set_title(f'Cols')
        
plt.suptitle('CA AFD linear separability', y=1);
plt.tight_layout()

### Distribution of the values of the cells of the data matrix

In [ ]:
t_plot = 0
t_plot = [t for t in range(T)]
t_plot = [0, T//2, -1]

bins = 50
val_min = 1
val_max = 100 # int or None

f, ax = plt.subplots(len(t_plot), 1, sharex=True, sharey=True, figsize=(10, 1.5 * len(t_plot)))

for i, t in enumerate(t_plot):
    values = X[t].flatten()
    values = values[values >= val_min]
    if val_max is not None:
        values = values[values < val_max]
    ax[i].hist(values, bins=bins)
    ax[i].set_title(f'time t = {t}')
    
f.suptitle('Histogram of the values of the cells of the data matrix over time');
plt.tight_layout();

In [ ]:
t_plot = 0
t_plot = [t for t in range(T)]
t_plot = [0, T//2, -1]

bins = 50
val_min = 0
val_max = None #int or None

f, ax = plt.subplots(len(t_plot), 1, sharex=True, sharey=True, figsize=(10, 1.5 * len(t_plot)))

for i, t in enumerate(t_plot):
    values = X[t].sum(0).flatten()
    values = values[values >= val_min]
    if val_max is not None:
        values = values[values < val_max]
    ax[i].hist(values, bins=bins)
    ax[i].set_title(f'time t = {t}')
    
f.suptitle('Histogram of the degrees of the nodes over time');
plt.tight_layout();

# Models

## HLBM

### Algo params

In [ ]:
t_ = 0
if X.ndim == 3:
    X_ = X[t_]
    Z_, W_ = Z[t_], W[t_]
else:
    X_ = X
    Z_, W_ = Z, W

sparse_X = True

frac_r, frac_c = .1, .1
frac_noise = 0.

lambda_0 = 3.
damping_factor = .7

n_init = 20
model_type = 'with_margins' # 'with_margins', 'without_margins'
estimated_margins = True # True, False
# 'skmeans' requires spherecluster & Python 3.7
init_type = 'kmeans' # 'skmeans', 'kmeans'
regularize_row, regularize_col = False, False
regularization_mode = 'all' # 'all' 'mixture'
em_type = 'CEM' # 'VEM', 'CEM'
dtype = 'float64'

n_init_clustering = 1
node_perturbation_rate = .1
multiplicative_init_rows = False # True, False
multiplicative_init_cols = False # True, False
power_multiplicative_init = 1 # True, False
given_Z, given_W = None, None
n_jobs = -1
random_state = None # or np.random.RandomState(42) 

max_iter = 200
tol_iter = 1e-8

min_float = 1e-15
min_proba_Z, min_proba_W = .05, .05
min_proba_mixture_proportions = 1e-2  # to avoid empty clusters
min_margin = 1e-10
min_gamma = 1e-8
threshold_absent_nodes = -1

# debug_list contains the names of the parameters fo the models
# or of the variational distribution that we wish to monitor
# during the fitting of the model
# This is done by writing the values of the model to disk
# so it takes time and space. Providing an empty list
# is the normal behavior
debug_list = []
debug_output = pathlib.Path(r'../dcblockmodels/model_debug_output')
verbose = 1
model_id = 1

### Initialization

In [ ]:
if sparse_X:
    if not sp.sparse.issparse(X_):
        X_ = general.to_sparse(X_)
else:
    if sp.sparse.issparse(X):
        X_ = general.to_dense(X_)

S_r = similarity_matrices.build_S(Z_, frac_r, frac_noise)
S_c = similarity_matrices.build_S(W_, frac_c, frac_noise)

S_r = sp.sparse.csr.csr_matrix(S_r)
S_c = sp.sparse.csr.csr_matrix(S_c)

### Fitting the model

In [ ]:
regularize_row, regularize_col = (lambda_0 != 0), (lambda_0 != 0)
lambda_r, lambda_c = lambda_0, lambda_0

model = HLBM(
    Kz=Kz, Kw=Kw,
    model_type=model_type,
    estimated_margins=estimated_margins,
    regularization_mode=regularization_mode,
    regularize_row=regularize_row,
    regularize_col=regularize_col,
    n_init=n_init,
    max_iter=max_iter,
    em_type=em_type,
    damping_factor=damping_factor,
    multiplicative_init_rows=multiplicative_init_rows,
    multiplicative_init_cols=multiplicative_init_cols,
    power_multiplicative_init=power_multiplicative_init,
    min_float=min_float,
    min_proba_Z=min_proba_Z,
    min_proba_W=min_proba_W,
    min_proba_mixture_proportions=min_proba_mixture_proportions,
    min_margin=min_margin,
    min_gamma=min_gamma,
    init_type=init_type,
    n_init_clustering=n_init_clustering,
    node_perturbation_rate=node_perturbation_rate,
    model_id=model_id,
    dtype=dtype,
    threshold_absent_nodes=threshold_absent_nodes,
    blockmodel_params=None,
    random_state=random_state,
    tol_iter=tol_iter,
    n_jobs=n_jobs,
    verbose=verbose, 
    debug_list=debug_list,
    debug_output=debug_output
)

# Fit model
model.fit(
    X_,
    given_Z=given_Z,
    given_W=given_W, 
    S_r=S_r, lambda_r=lambda_r,
    S_c=S_c, lambda_c=lambda_c
)

## Load/save model

### Save

In [ ]:
model.save(path='../saved_models', modelname='my_model')

### Load

In [ ]:
model = general.load_model('../saved_models/my_model')
model

# Metrics & visualizations

## Partitions and criterion

In [ ]:
Z_model, W_model = model.best_partition(mode='likelihood', n_first=1)[0]
#Z_model, W_model = model.best_partition(mode='consensus: hbgf', n_first=(model.n_init) // 2)[0]

plot.plot_criterions(
    model,
    thr_decrease=1000,
    i_start=0, i_end=-1,
    legend=True
)

## HLBM

### Metrics

In [ ]:
Z_model, W_model = model.best_partition(mode='likelihood', n_first=1)[0]

metrics.print_metrics(
    Z_model, W_model, Z_, W_,
    absent_nodes=None,
    print_each_timestep=False
)

### Distribution of the metrics

In [ ]:
f, ax = plt.subplots(figsize=(8, 4.5))

caris = np.array([
    metrics.get_metrics(
        Z_model, W_model,
        Z_, W_,
        absent_nodes=None
    )['cari']
    for Z_model, W_model in model.best_partition(mode='likelihood', n_first=model.n_init)
])
sns.kdeplot(data=caris, ax=ax, bw=.2, clip=(caris.min() - .1, caris.max() + .1));
ax.set_title(f'{model.__class__.__name__}: max CARI = {100 * caris.max():.2f}');
ax.set_xlabel('global CARI values');

### Confusion matrix

In [ ]:
print('   Z')
print(
    metrics.cmat_clustering(
        confusion_matrix(Z_model, Z_)
    ),
    end='\n\n'
)
print('   W')
print(
    metrics.cmat_clustering(
        confusion_matrix(W_model, W_)
    ),
    end='\n\n'
)

## Model parameters

`model.best_parameters` : a list of length the number of initializations of the model. Each element of the list is a tuple in the form `(crit, param_dic)`, where `crit` is the best value of the objective criterion of the model of the given init and `param_dic` contains the parameters of the model that gave this `crit` 

In [ ]:
model.best_parameters[0]

Mapping the indexes of the found clusters to the indexes of the true clusters using the Kuhn Munkres/Hungarian algorithm on the confusion matrix

In [ ]:
from scipy.optimize import linear_sum_assignment
from sklearn.metrics import confusion_matrix

Z_model, W_model = model.best_partition(mode='likelihood', n_first=1)[0]

cmat_Z = confusion_matrix(Z_model, Z_)
cmat_W = confusion_matrix(W_model, W_)
    
indexes_Z = linear_sum_assignment(-cmat_Z)[1]
indexes_W = linear_sum_assignment(-cmat_W)[1]

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(12, 6))

gamma_model = model.best_parameters[0][1]['gamma']
print(gamma_model)
print(indexes_W)
print(indexes_Z)
print(Z_model)
print(Z_)
print(W_model)
print(W_)
print(cmat_Z)
print(cmat_W)

reordered_gamma_model = gamma_model[np.ix_(indexes_Z, indexes_W)]

sns.heatmap(reordered_gamma_model, ax=ax[0],  square=True)
sns.heatmap(gamma[0], ax=ax[1], square=True)

ax[0].set_title('Estimated gamma');
ax[1].set_title('True gamma');